<h1> DattoRMM - Audit Log Scrape and Ingest </h1>

In [ ]:
# general purpose
import time
import datetime as dt

# Https Traffic
import requests
import socket
from urllib3.connection import HTTPSConnection

# Spider
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import selenium
import scrapy
import re
from bs4 import BeautifulSoup
import lxml

# Data Ingest and Visual
import pandas as pd
[REDACTED]/.pyplot as plt
import seaborn as sns

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# Create Bot Object

## Create Firefox Driver

In [ ]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

firefox_dev_binary = FirefoxBinary(r'C:/Program Files/Firefox Developer Edition/firefox.exe')
f_driver = webdriver.Firefox(firefox_binary=firefox_dev_binary, executable_path=r'D:/web_drivers/mozilla/geckodriver-v0.31.0-win64/geckodriver.exe')

## Create Chrome Driver

## Request Header Info (if needed to impersonate api browser calls)

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://concord.centrastage.net'
auth_url = f'https://auth.datto.com'
#home_url = f'{baseUrl}'
links = []

# Login to DattoRMM

In [ ]:
f_driver.get(auth_url)

In [ ]:
title = f_driver.find_element_by_xpath('//h1')
title.text

In [ ]:
login_txtbox = f_driver.find_element_by_xpath("//input[@id='form_username']")
login_txtbox.send_keys((config['dattormm']['username']),Keys.ENTER)

In [ ]:
pass_txtbox = f_driver.find_element_by_xpath("//input[@id='form_password']")
pass_txtbox.send_keys((config['dattormm']['password']),Keys.ENTER)

In [ ]:
time.sleep(3)

In [ ]:
mfa_txtbox = f_driver.find_element_by_id('authy_token')
mfa_txtbox.send_keys('715082', Keys.ENTER) # number entered manually for now /TODO: automate MFA

# Create Policy DataFrames

## Scrape dropdown information for child policy pages

In [ ]:
sites_info_list = []
sites_info_dict = {}
sites_dropdown = f_driver.find_element_by_xpath("//div[@id='profiles-tab']")

#object of ActionChains
a = ActionChains(f_driver)

mouseover_sites_dropdown = a.move_to_element(sites_dropdown)
mouseover_sites_dropdown.perform()
time.sleep(1)
sites_profiles_menu = f_driver.find_element_by_xpath("//div[@id='menuProfilesDiv']")
df_sites_info = pd.DataFrame()
df_sites_info_list = []

for li in sites_profiles_menu.find_elements_by_xpath('//a'):
    attribute = li.get_attribute('title')
    if None:
        continue
    elif attribute:
        df_sites_info_list.append(attribute)
site_profiles_prog = re.compile(r'[\d]+$')
site_profiles_list = []
for a in df_sites_info_list:
    site_profile = {}
    site_profile['siteName'] = a

    for element in (f_driver.find_elements_by_xpath(f"//a[@title='{a}']")):
        href = (element.get_attribute('href'))
        site_profile['href'] = (site_profiles_prog.findall(href)[0])

    site_profiles_list.append(site_profile)

## Define Table Heading Configuration /override/nooverride/

In [ ]:
no_policy_override = ['policyName','policyTargets','policyType','policyBlank','policyStatus','policyEnd']
yes_policy_override = ['policyOverride','policyName','policyTargets','policyType','viewOverride','policyBlank','policyStatus','policyEnd']

## Breakout Account Policies - Main Page

### Main Page Policy

In [ ]:
f_driver.get(f'{baseUrl}/csm/policy/list')

In [ ]:
policies_heading = f_driver.find_elements_by_xpath("//h1")
description_prog = re.compile(r'[^\r\n]+')
policies_titles = []
matches = enumerate(policies_heading)
for index, match in matches:
    policies_titles.append(description_prog.findall(match.text))
for title in policies_titles:
    if title != []:
        policies_title = title

In [ ]:
policy_title_prog = re.compile((r'[^\:]+'))
policy_title = policy_title_prog.findall(str(policies_title[0]))

In [ ]:
account_policies_list = []
policy_table = f_driver.find_elements_by_xpath("//tbody[@id='policyListTable']//td")
pol_iter_counts = len(policy_table)/4

policy_table_label_iteration = []
policy_table_labels = no_policy_override
for x in range(0,int(pol_iter_counts),1):
    for p in policy_table_labels:
        policy_table_label_iteration.append(p)

In [ ]:
policy_dicts = {}
policy_df_list = []
targets_prog = re.compile(r'[^\,]+')
for label in range(0, len(policy_table)):
    policy_dict = {}
    try:
        if policy_table_label_iteration[label] == 'policyBlank':
            continue
        elif policy_table_label_iteration[label] == 'policyEnd':
            policy_df_list.append(policy_dicts)
            policy_dicts = {}

        elif policy_table_label_iteration[label] == 'policyTargets':
            policy_targets_list = []
            for target in targets_prog.findall(policy_table[label].text):
                policy_targets_list.append(target)
            policy_dict[policy_table_label_iteration[label]] = policy_targets_list
            policy_dicts.update(policy_dict)
        else:
            policy_dict[policy_table_label_iteration[label]] = policy_table[label].text
            policy_dicts.update(policy_dict)
    except Exception as e:
        print(e)

In [ ]:
df = pd.DataFrame(policy_df_list)

In [ ]:
df['policyLevel'] = policy_title[0]
df['policyTableName'] = policy_title[1]

In [ ]:
df

## Break out Site Policies Page

### Site Policy Iterations

In [ ]:
f_driver.find_elements_by_xpath()

In [ ]:
site_profiles_list[0]

### Find Site Policy Title

In [ ]:
f_driver.get(f"{baseUrl}/csm/policy/list/{site_profiles_list[1]['href']}")

In [ ]:
policy_section_prog = re.compile(r'([A-Z]+\s[A-Z]+){1}$')
policy_section_table_objects = []

In [ ]:
policies_heading = f_driver.find_elements_by_xpath("//h1")
description_prog = re.compile(r'[^\r\n]+')
policies_titles = []
matches = enumerate(policies_heading)
for index, match in matches:
    policies_titles.append(description_prog.findall(match.text))
for title in policies_titles:
    if title != []:
        policies_title = title

In [ ]:
print(policies_title)

### Find Site Policy Tables (Account and Site /custom/)

In [ ]:
table_headings = f_driver.find_elements_by_xpath('//h2[contains(@onclick,"Policy")]')

### Expand policy tables if needed

In [ ]:
for h2 in table_headings:
    try:
        h2.find_element_by_xpath("//span[contains(@class,'collapsed')]").click()
    except:
        continue

## Site Account Policies to DataFrame

In [ ]:
account_policies_list = []

description_prog = re.compile(r'[^\n\r]+')
tables = f_driver.find_elements_by_xpath(('//tbody[contains(@id,"policyListTable")]'))

In [ ]:
tables

In [ ]:
account_table = tables[0].find_elements_by_xpath('//tr[contains(@id,"policyRow")]')

In [ ]:

try:
    print(f_driver.find_element_by_xpath("//img[@title='Override active']"))
    policy_table_labels = yes_policy_override
except:
    policy_table_labels = no_policy_override

policy_table_label_iteration = []
pol_iter_counts = len(account_table)/len(policy_table_labels)


for x in range(0,int(pol_iter_counts)+1):
    for p in policy_table_labels:
        policy_table_label_iteration.append(p)

In [ ]:
policy_table_label_iteration

In [ ]:
b_list = []
for table in account_table:
    a = (table.find_elements_by_xpath('//a[@class="policy-link"]'))
    for x in a:

        print(x.text)
        print(x.get_attribute('href'))
        print('*'*5)
    print(a.find_element_by_xpath("//td").text)



In [ ]:
policy_dicts = {}
policy_df_list = []
targets_prog = re.compile(r'[^\,]+')
for table in range(0, len(account_table)):
    policy_dict = {}
    label_name = policy_table_label_iteration[table]
    label_value = account_table[table]
    print(table)
    print(f'{label_name}:{label_value.text}')
    try:
        if (label_name == 'policyBlank') | (label_name == 'viewOverride'):
            continue
        elif label_name == 'policyEnd':
            policy_df_list.append(policy_dicts)
            policy_dicts = {}
        elif label_name == 'policyOverride':
           try:
               policy_dict[label_name] = (label_value.find_element_by_xpath("./img").get_attribute('title'))
               policy_dicts.update(policy_dict)
           except:
               continue
        elif policy_table_label_iteration[label] == 'policyTargets':
            policy_targets_list = []
            for target in targets_prog.findall(label_value.text):
                policy_targets_list.append(target)
            policy_dict[label_name] = policy_targets_list
            policy_dicts.update(policy_dict)
        else:
            #print(policy_table[label].text)
            policy_dict[label_name] = label_value.text
            policy_dicts.update(policy_dict)
    except Exception as e:
        print(e)
print(policy_dicts)

In [ ]:
policy_df_list

In [ ]:
df = pd.DataFrame(policy_df_list)
df['policyLevel'] = policy_title[0]
df['policyTableName'] = policy_title[1]

In [ ]:
df

In [ ]:
#for table in f_driver.find_elements_by_xpath('//h2'):
for element in f_driver.find_elements_by_xpath("//div[contains(@id,'PoliciesDiv')]"):
    print(element.text)
    #print('inner.inner'*10)

In [ ]:
policy_dicts = {}
policy_df_list = []
targets_prog = re.compile(r'[^\,]+')
for label in range(0, len(policy_table)):
    policy_dict = {}
    try:
        if policy_table_label_iteration[label] == 'policyBlank':
            continue
        elif policy_table_label_iteration[label] == 'policyEnd':
            policy_df_list.append(policy_dicts)
            policy_dicts = {}

        elif policy_table_label_iteration[label] == 'policyTargets':
            policy_targets_list = []
            for target in targets_prog.findall(policy_table[label].text):
                policy_targets_list.append(target)
            policy_dict[policy_table_label_iteration[label]] = policy_targets_list
            policy_dicts.update(policy_dict)
        else:
            policy_dict[policy_table_label_iteration[label]] = policy_table[label].text
            policy_dicts.update(policy_dict)
    except Exception as e:
        print(e)

In [ ]:
policy_section_table_objects

In [ ]:
account_patches = f_driver.get('https://concord.centrastage.net/csm/system/patches')

In [ ]:
description_prog = re.compile(r'[^\r\n]+')
description_prog.findall()
df_list = []
matches = enumerate(description_matches)
for index, match in matches:
    print(index, (description_prog.findall(match.text)))
    #itemlist = (description_prog.findall(match.text))
    if len(itemlist) == 3:
        df_list.append({'description': itemlist[0], 'price': itemlist[1], 'location': itemlist[2]})
    elif len(itemlist) == 4:
        df_list.append(
            {'description': itemlist[0], 'price': itemlist[1], 'milage': itemlist[2], 'location': itemlist[3]})
    else:
        continue

In [ ]:
f_driver.get(website)

In [ ]:
next_link = f_driver.find_element_by_xpath("//a[@class='nextLink']")

In [ ]:
next_url = next_link.get_attribute('href')

In [ ]:
f_driver.get(next_url)

In [ ]:
next_link.click()

# Quit Drivers

In [ ]:
f_driver.quit()
c_driver.quit()